# 第二章：隐函数和自动微分有效区分定点
## 定点求解器
让我们从数值固定点开始，比如那些作为深度均衡模型 (DEQs) 基础的固定点。 我们的主要目标是解释如何对由不动点方程隐式定义的函数执行有效的自动微分。  
在数学上，对于某些函数$f:R^{n}→R^{n}$我们说 
$z∈R^{n}$ 是$ f$ 的不动点，如果$$z=f(z)$$  考虑固定点的一种方法是，如果我们应用 $f$，我们就会留在同一个地方。许多迭代算法，如优化算法，都适合这种模式。 因此，我们在这里为不动点开发的高效自动微分方法可以立即应用于优化例程的高效微分，只需选择$ f$ 作为相关优化算法的更新^数即可。  
考虑不动点方程的另一种方法是将其作为（非线性）方程的系统，就像我们通常可以指定为$g（z）= 0$一样，只不过这里两边都有$z$。 事实上，如果我们选择$ g(z)=f(z)−z $那么我们可以简化为非线性方程的通常形式。  
更一般地说，我们可能有一个*参数化函数* $f:R^{p}×R^{n}→R^{n}$，它采用参数向量$ a∈R^{p}$。 然后我们写一个定点为$$z = f(a, z)$$.  
这对应于具有参数化的方程组。 一旦涉及到参数，我们可能会问一些问题，例如随着我们改变参数值，固定点会如何变化。 但在进入此类问题之前，让我们先看看如何计算数值不动点。  
也许尝试计算数值不动点的最简单方法是朴素[前向迭代](https://en.wikipedia.org/wiki/Fixed-point_iteration) 我们迭代 $z_{k+1} = f(z_k) $直到$z_{k+1}$ 与 $z_k$保持足够接近：


In [1]:
import jax.numpy as jnp

def fwd_solver(f, z_init):
    z_prev, z = z_init, f(z_init)
    while jnp.linalg.norm(z_prev - z) > 1e-5:
        z_prev, z = z, f(z)
    return z

这个方法是否成功取决于我们如何初始化它以及 $f$ 的属性（尤其是$f$是否有任何不动点！）。 即使它确实成功了，也可能需要很多步骤才能收敛。  
一个更复杂的方法是[牛顿迭代](https://en.wikipedia.org/wiki/Newton%27s_method#k_variables,_k_functions)，我们使用关于$f$的导数信息来采取更智能的步骤（以每步使用更多计算为代价）：


In [ ]:
import jax

def newton_solver(f, z_init):
  f_root = lambda z: f(z) - z
  g = lambda z: z - jnp.linalg.solve(jax.jacobian(f_root)(z), f_root(z))
  return fwd_solver(g, z_init)

最后，第三种方法是[安德森加速](https://en.wikipedia.org/wiki/Anderson_acceleration)：

In [ ]:
def anderson_solver(f, z_init, m=5, lam=1e-4, max_iter=50, tol=1e-5, beta=1.0):
  x0 = z_init
  x1 = f(x0)
  x2 = f(x1)
  X = jnp.concatenate([jnp.stack([x0, x1]), jnp.zeros((m - 2, *jnp.shape(x0)))])
  F = jnp.concatenate([jnp.stack([x1, x2]), jnp.zeros((m - 2, *jnp.shape(x0)))])

  res = []
  for k in range(2, max_iter):
    n = min(k, m)
    G = F[:n] - X[:n]
    GTG = jnp.tensordot(G, G, [list(range(1, G.ndim))] * 2)
    H = jnp.block([[jnp.zeros((1, 1)), jnp.ones((1, n))],
                   [ jnp.ones((n, 1)), GTG]]) + lam * jnp.eye(n + 1)
    alpha = jnp.linalg.solve(H, jnp.zeros(n+1).at[0].set(1))[1:]

    xk = beta * jnp.dot(alpha, F[:n]) + (1-beta) * jnp.dot(alpha, X[:n])
    X = X.at[k % m].set(xk)
    F = F.at[k % m].set(f(xk))

    res = jnp.linalg.norm(F[k % m] - X[k % m]) / (1e-5 + jnp.linalg.norm(F[k % m]))
    if res < tol:
      break
  return xk

有了这些求解器，我们就可以计算一些不动点了！ 我们将看一个玩具 DEQ 定点层，它可以使用我们开发的任何求解器：

In [ ]:
def fixed_point_layer(solver, f, params, x):
  z_star = solver(lambda z: f(params, x, z), z_init=jnp.zeros_like(x))
  return z_star

该层作为参数  
1.一个求解器，恰如其分地命名为`solver`,<br />
2.要找到固定点的函数`f`, <br />
3.参数`params`,<br />
4.输入值`x` 


使用这些输入，它计算函数的定点作为其输出。
这是我们将使用的函数`f`，因此它的参数只是一个权重矩阵`W`：

In [ ]:
f = lambda W, x, z: jnp.tanh(jnp.dot(W, z) + x)

我们将为参数和输入初始化一些随机值：

In [ ]:
from jax import random

ndim = 10
W = random.normal(random.PRNGKey(0), (ndim, ndim)) / jnp.sqrt(ndim)
x = random.normal(random.PRNGKey(1), (ndim,))

In [ ]:
现在，我们可以使用求解器找到不动点了：

In [ ]:
z_star = fixed_point_layer(fwd_solver, f, W, x)
print(z_star)

In [ ]:
z_star = fixed_point_layer(newton_solver, f, W, x)
print(z_star)

In [ ]:
z_star = fixed_point_layer(anderson_solver, f, W, x)
print(z_star)

直到数值容差，我们可以看到我们正在计算相同的数值不动点，而不管我们使用什么求解器来找到它。 也就是说，我们将要计算的值与计算方法解耦。
## 通过迭代求解器进行简单的自动微分
我们已经可以通过 `fixed_point_layer` 函数进行区分。 例如，我们可以使用 `jax.grad `计算输出总和相对于参数矩阵的梯度:

In [ ]:
g = jax.grad(lambda W: fixed_point_layer(fwd_solver, f, W, x).sum())(W)
print(g[0])

如我们所料，在不超过数值公差的情况下，此梯度值不取决于固定点的计算方法：

In [ ]:
g = jax.grad(lambda W: fixed_point_layer(newton_solver, f, W, x).sum())(W)
print(g[0])

如果我们已经可以通过这些定点层进行区分，那还有什么可做的呢？  
问题是这种区分方式的效率极低。 我们通过求解器的所有展开迭代进行区分。 对于每一步，我们的自动微分工具都会存储前向传播的值，以便在后向传播中消耗。 这意味着使用大量内存！  
为了做得更好，我们需要从隐函数微分的角度来思考。
## 隐函数定理
首先，让我们回顾一些基础知识并设置我们的微分符号。 对于函数 $f : \mathbb R^n \to \mathbb R^m$，我们将在点$x \in \mathbb R^n$处求得的导数表示为$$\partial f(x) : \mathbb R^n \to \mathbb R^m.$$
换句话说，$\partial f(x)$ 也是一个函数，从输入空间$\mathbb R^n$上的扰动映射到输出空间$\mathbb R^m$上的扰动。 此外，$\partial f(x)$是线性的。 我们可以认为在$x$处为$f$编写一阶泰勒级数为$$f(x + v) = f(x) + \partial f(x) v + \mathcal O(\|v\|^2),$$
其中$v \in \mathbb R^n$是一个扰动向量。  
请注意，因为$\partial f(x)$是线性的，所以我们使用并列而不是括号将其对向量$v$的应用表示为$\partial f(x) v$。 这很方便，并且强调我们还可以将$\partial f(x)$视为一个矩阵（相对于输入和输出空间的标准基）：$$\partial f(x) \in \mathbb R^{m \times n}$$
在本节中，我们主要将$\partial f(x)$作为矩阵进行操作。  
最后一点符号用于多元函数。 当我们只想区分一个参数时，将在$\partial$运算符上使用下标：$$\begin{align}
\partial_0 f(x, y) &\triangleq \partial g(x) \text{ where } g(x) = f(x, y),
\\
\partial_1 f(x, y) &\triangleq \partial g(y) \text{ where } g(y) = f(x, y).
\end{align}$$
此符号基于 Spivak 的经典流形微积分 (1965)，也用于 Sussman 和 Wisdom 的[经典力学结构和解释](https://mitpress.mit.edu/sites/default/files/titles/content/sicm_edition_2/book.html) (2015) 和[函数微分几何](https://mitpress.mit.edu/books/functional-differential-geometry) (2013)。 后两本书都是开放获取的。 请特别参阅函数微分几何的序言以捍卫这种符号。 这种表示法的另一个优点是它与 JAX 的自动微分 API 直接对应！  
现在我们准备说明我们将在这些笔记中使用的[隐函数定理](https://en.wikipedia.org/wiki/Implicit_function_theorem)的版本！
```
隐函数定理。令 $f : \mathbb R^p \times \mathbb R^n \to \mathbb R^n$ 且$ a_0 \in \mathbb R^p$,$z_0 \in \mathbb R^n$ 使得
1.$f(a_0, z_0) = 0$,且  
2.$f(a, z^*(a)) = 0 \quad \forall a \in S_{a_0}$,且  
3.$z^*$ 在$S_{a_0}$上是可微的。  
请参阅 Krantz 和 Parks 所著的 * The Implicit Function Theorem: History, Theory, and Applications (2002) * 一书，了解详细的历史以及[关于隐函数定理的许多变体的注释](http://www.its.caltech.edu/~kcborder/Notes/IFT.pdf)。
```
我们可以将$f(a, z) = 0$视为在$z$上定义非线性方程组，由$a$参数化。 点$(a_0, z_0)$是标称解。 该定理之所以强大，是因为它告诉我们，我们可以想到一个解映射函数$z^*$，它满足$$f(a, z^*(a)) = 0 \quad \forall a \in S_{a_0}.$$
注意两边都是$a$的函数，所以我们可以对$a$的两边进行微分，并在点$(a_0, z_0)$处求值得到$$\partial_0 f(a_0, z_0) + \partial_1 f(a_0, z_0) \partial z^*(a_0) = 0.$$
重新排列，我们发现解映射的雅可比矩阵必须由下式给出
$$\partial z^*(a) = - [ \partial_1 f(a_0, z_0)]^{-1} \partial_0 f(a_0, z_0).$$
也就是说，解映射的雅可比矩阵可以用解点$(a_0, z_0)$处$f$的雅可比矩阵来表示。 换句话说，给定$a_0$，无论我们如何求解方程来计算$z_0$，我们仍然可以仅使用解点处的导数信息来计算雅可比行列式。 这可以帮助我们避免通过迭代求解器区分！  
我们也可以将其调整为固定点。 隐函数定理立即适用，因为我们可以通过$g(a, z) \triangleq f(a, z) - z$将我们的不动点$z = f(a, z)$转换为非线性系统$g(a, z) = 0$，但我们可以重做上面的导数计算来得到一个很好的表达式。  
为了得到一个很好的定点解映射雅可比表达式，我们从一个定点开始$$z_0 = f(a_0, z_0),$$
然后我们考虑局部解映射$z^∗$，它满足$$z^*(a) = f(a, z^*(a))$$
作为$a$左右$a_0$的函数。 然后我们区分双方并在$(a_0，z_0)$处进行评估，$$\partial z^*(a_0) = \partial_0 f(a_0, z_0) + \partial_1 f(a_0, z_0) \partial z^*(a_0),$$
并重新排列以获得$$\partial z^*(a_0) = [ I - \partial_1 f(a_0, z_0)]^{-1} \partial_1 f(a_0, z_0).$$
同样，我们只使用函数$f$在不动点本身的导数信息，就有了定点解的导数表达式。 我们可以节省内存！  
现在我们有了定点解图导数的数学表达式，我们只需要将它连接到我们的自动微分系统。